### Install dependencies


In [ ]:
!pip install onnx
!pip install onnxruntime

import torch.onnx
import onnx
import onnxruntime

### Load 'onnx' file

In [ ]:
# load the onnx format file
onnx_model = onnx.load('/content/drive/MyDrive/UCI/output.onnx')  # from model_compression.ipynb

In [ ]:
# test for 'onnx' format file
# onnx_model.graph.node[0].input
# onnx_model.graph.node[0].output

['/features/features.0/Conv_output_0']

### Graph

In [ ]:
# Node class definition
class Node:
    def __init__(self, name, node_type, inputs, outputs):
        self.name = name  # str
        self.node_type = node_type  # str
        self.inputs = inputs  # List[str]
        self.outputs = outputs  # List[str]
        self.parent = []  # parent node names
        self.child_nodes = []  # child node names

    def __repr__(self):
        return f"node_type={self.node_type}, parent={', '.join(self.parent)}, child_nodes={', '.join(self.child_nodes)}"


In [ ]:
node_dict = {}

# Create all the nodes from onnx file
for node in onnx_model.graph.node:
    node_name = node.name
    node_type = node.op_type
    node_inputs = list(node.input)
    node_outputs = list(node.output)

    node_dict[node_name] = Node(node_name, node_type, node_inputs, node_outputs)  # create Node instance

# Find dependencies each other
for node in onnx_model.graph.node:
    current_node = node_dict[node.name]
    # Check one's output with other's input
    for output_name in node.output:
        for another_node in onnx_model.graph.node:
            if output_name in another_node.input:
                # If matched
                if another_node.name in node_dict:
                    # append to the list
                    current_node.child_nodes.append(another_node.name)  # for child nodes
                    node_dict[another_node.name].parent.append(current_node.name)  # for parent nodes

node_dict  # key : node_name / value : Node instance

{'/features/features.0/Conv': node_type=Conv, parent=, child_nodes=/features/features.1/Relu,
 '/features/features.1/Relu': node_type=Relu, parent=/features/features.0/Conv, child_nodes=/features/features.2/Conv,
 '/features/features.2/Conv': node_type=Conv, parent=/features/features.1/Relu, child_nodes=/features/features.3/Relu,
 '/features/features.3/Relu': node_type=Relu, parent=/features/features.2/Conv, child_nodes=/features/features.4/MaxPool,
 '/features/features.4/MaxPool': node_type=MaxPool, parent=/features/features.3/Relu, child_nodes=/features/features.5/Conv,
 '/features/features.5/Conv': node_type=Conv, parent=/features/features.4/MaxPool, child_nodes=/features/features.6/Relu,
 '/features/features.6/Relu': node_type=Relu, parent=/features/features.5/Conv, child_nodes=/features/features.7/Conv,
 '/features/features.7/Conv': node_type=Conv, parent=/features/features.6/Relu, child_nodes=/features/features.8/Relu,
 '/features/features.8/Relu': node_type=Relu, parent=/feature

In [ ]:
att = ['name', 'node_type', 'inputs', 'outputs', 'parent', 'child_nodes']  # define in the Node class

# test for Conv0
for node in att:
    print(f"{node}: {getattr(node_dict[onnx_model.graph.node[0].name], node)}")

name: /features/features.0/Conv
node_type: Conv
inputs: ['input.1', 'features.0.weight', 'features.0.bias']
outputs: ['/features/features.0/Conv_output_0']
parent: []
child_nodes: ['/features/features.1/Relu']


In [ ]:
# Node class definition
# class Node:
#     def __init__(self, name, node_type):
#         self.name = name  # str
#         self.node_type = node_type  # str
#         self.parent = [] # parent node, list of nodes if there are multiple inputs
#         self.child_nodes = []

#     def __repr__(self):
#         return f"node_type={self.node_type}, parent={', '.join([node.name for node in self.parent])}, child_nodes={', '.join([node.name for node in self.child_nodes])}"

# node_dict = {}  # key : node_name

# # First, create all the nodes
# for node in onnx_model.graph.node:
#     node_name = node.output[0]  # use the first output name as the node name
#     node_type = node.op_type  # str
#     node_dict[node_name] = Node(node_name, node_type)

# # Next, link the nodes together
# for node in onnx_model.graph.node:
#     current_node = node_dict[node.output[0]]  # use the first output name as the node name
#     for input_name in node.input:
#         if input_name in node_dict:
#             parent_node = node_dict[input_name]
#             parent_node.child_nodes.append(current_node)
#             current_node.parent.append(parent_node)

# # print node_dict
# node_dict
